In [13]:
# 4. Second Turn: Continue the Thread

question_2 = "How can I filter and view this specific conversation in the Langsmith UI?"

# PASS THE QUESTIONS BY NAME and the SAME unique_thread_id in langsmith_extra metadata
ai_answer_2 = langsmith_conversational_rag(
    question=question_2, 
    langsmith_extra={"metadata": {"conversation_id": unique_thread_id}}
)

print(f"\n--- Question 2 ---\n{question_2}")
print(f"\n--- Answer 2 ---\n{ai_answer_2}")

print("\n\nTwo traces were created with the same 'conversation_id'. Check Langsmith!")


--- Question 2 ---
How can I filter and view this specific conversation in the Langsmith UI?

--- Answer 2 ---
I don't know how to filter and view a specific conversation in the Langsmith UI. The provided context only discusses the @traceable decorator and adding metadata to traces. It does not mention filtering or viewing conversations in the Langsmith UI.


Two traces were created with the same 'conversation_id'. Check Langsmith!


In [10]:
# 3. First Turn: Start the Conversation Thread (TWEAK)

question_1 = "How do I trace my conversation history using Langsmith?"

# TWEAK: Pass the unique_thread_id in langsmith_extra metadata
ai_answer_1 = langsmith_conversational_rag(
    question=question_1, 
    langsmith_extra={"metadata": {"conversation_id": unique_thread_id}}
)

print(f"--- Question 1 ---\n{question_1}")
print(f"\n--- Answer 1 ---\n{ai_answer_1}")

--- Question 1 ---
How do I trace my conversation history using Langsmith?

--- Answer 1 ---
You can use the @traceable decorator to log traces of your functions, including conversations, in LangSmith. To track conversation history, add the decorator to the relevant functions and optionally pass a 'metadata' dictionary for additional context. This will automatically track inputs, outputs, and errors, providing a record of your conversation history.


In [11]:
# 2. RAG Pipeline Functions

@traceable(run_type="retriever", name="Retrieve Documents")
def retrieve_documents(question: str):
    # This invokes the retriever defined in utils.py
    return retriever.invoke(question)   

@traceable(run_type="llm", name="Final LLM Call (Groq)")
def call_llm(messages: List[dict]):
    # This LLM call is automatically traced by LangChain/Groq
    return llm_client.invoke(messages)

@traceable(run_type="chain", name="RAG Response Generation")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {"role": "system", "content": RAG_SYSTEM_PROMPT},
        {"role": "user", "content": f"Context: {formatted_docs} \n\n Question: {question}"}
    ]
    response = call_llm(messages)
    return response

@traceable(run_type="chain", name="Root Conversational RAG")
def langsmith_conversational_rag(question: str, **kwargs):
    # kwargs will contain the langsmith_extra metadata
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.content

In [12]:
# 1. Setup Environment, Imports, and UUID
import os
from dotenv import load_dotenv
from langsmith import traceable 
from langchain_groq import ChatGroq 
from typing import List
import nest_asyncio
import uuid
import warnings # New import to handle warnings
from utils import get_vector_db_retriever, RAG_SYSTEM_PROMPT

# Suppress the specific LangChain Deprecation Warning
warnings.filterwarnings("ignore", category=DeprecationWarning, module="langchain")

# --- Configuration ---
load_dotenv()
MODEL_NAME = "llama-3.3-70b-versatile" # Using the confirmed, working Groq model
llm_client = ChatGroq(model=MODEL_NAME)
nest_asyncio.apply()
retriever = get_vector_db_retriever()

# TWEAK: Generate a unique thread ID for the entire conversation
unique_thread_id = str(uuid.uuid4())
print(f"Generated Unique Thread ID: {unique_thread_id}")

Generated Unique Thread ID: 54483e69-9497-4f5f-a7ed-bc00d34bd039
